In [2]:
from config import env
from langchain_neo4j import Neo4jGraph

In [49]:
url = "bolt://localhost:7687"
username = "neo4j"
password = "password"

graph = Neo4jGraph(url=url, username=username, password=password)

In [55]:
from langchain_experimental.graph_transformers import LLMGraphTransformer
from langchain_aws import ChatBedrock

llm = ChatBedrock(
    temperature=0,
    model="us.anthropic.claude-3-5-sonnet-20240620-v1:0",
    region=env.AWS_REGION,
    aws_access_key_id=env.AWS_ACCESS_KEY_ID,
    aws_secret_access_key=env.AWS_SECRET_ACCESS_KEY,
)
llm_transformer = LLMGraphTransformer(llm=llm)

In [51]:
from langchain_community.document_loaders import WikipediaLoader

query = "Warren Buffett"
raw_documents = WikipediaLoader(query=query).load()

In [56]:
graph_documents = llm_transformer.convert_to_graph_documents(raw_documents)
print(f"Nodes:{graph_documents[0].nodes}")
print(f"Relationships:{graph_documents[0].relationships}")

Error raised by bedrock service
Traceback (most recent call last):
  File "C:\Users\prode\PycharmProjects\chat-pdf-GraphRAG-neo4j-langchain-bedrock\.venv\Lib\site-packages\langchain_aws\llms\bedrock.py", line 941, in _prepare_input_and_invoke
    response = self.client.invoke_model(**request_options)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\prode\PycharmProjects\chat-pdf-GraphRAG-neo4j-langchain-bedrock\.venv\Lib\site-packages\botocore\client.py", line 601, in _api_call
    return self._make_api_call(operation_name, kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\prode\PycharmProjects\chat-pdf-GraphRAG-neo4j-langchain-bedrock\.venv\Lib\site-packages\botocore\context.py", line 123, in wrapper
    return func(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\prode\PycharmProjects\chat-pdf-GraphRAG-neo4j-langchain-bedrock\.venv\Lib\site-packages\botocore\client.py", line 1074, in _make_api_call
    raise e

ThrottlingException: An error occurred (ThrottlingException) when calling the InvokeModel operation (reached max retries: 4): Too many requests, please wait before trying again.

In [ ]:
graph.add_graph_documents(graph_documents, include_source=True)

# Query the graph

In [47]:
from langchain_neo4j import GraphCypherQAChain
from langchain_aws import ChatBedrock

chat_bedrock_model = ChatBedrock(
    temperature=0,
    model="us.anthropic.claude-3-5-sonnet-20240620-v1:0",
    region=env.AWS_REGION,
    aws_access_key_id=env.AWS_ACCESS_KEY_ID,
    aws_secret_access_key=env.AWS_SECRET_ACCESS_KEY,
)

chain = GraphCypherQAChain.from_llm(
    cypher_llm=chat_bedrock_model,
    qa_llm=chat_bedrock_model,
    graph=graph,
    verbose=True,
    allow_dangerous_requests=True,
)

In [48]:
chain.invoke("Which university did Warren Buffett attend?")



> Entering new GraphCypherQAChain chain...


Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.UnknownPropertyKeyWarning} {category: UNRECOGNIZED} {title: The provided property key is not in the database} {description: One of the property names in your query is not available in the database, make sure you didn't misspell it or that the label is available when you run this statement in your application (the missing property name is: name)} {position: line: 1, column: 18, offset: 17} for query: 'MATCH (p:Person {name: "Warren Buffett"})-[:EDUCATED_AT]->(o:Organization)\nWHERE o.name CONTAINS "University"\nRETURN o.name'
Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.UnknownPropertyKeyWarning} {category: UNRECOGNIZED} {title: The provided property key is not in the database} {description: One of the property names in your query is not available in the database, make sure you didn't misspell it or that the label is available when yo

Generated Cypher:
MATCH (p:Person {name: "Warren Buffett"})-[:EDUCATED_AT]->(o:Organization)
WHERE o.name CONTAINS "University"
RETURN o.name
Full Context:
[]

> Finished chain.


{'query': 'Which university did Warren Buffett attend?',
 'result': "I apologize, but I don't have any specific information about which university Warren Buffett attended. Without reliable data on this topic, I can't provide an accurate answer to your question."}